In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from IPython import get_ipython
from IPython.display import display
from datasets import load_dataset
import pandas as pd

ds = pd.read_json("hf://datasets/Amod/mental_health_counseling_conversations/combined_dataset.json", lines=True)
ds

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import pipeline
import torch

model_name = "meta-llama/Llama-3.2-1B-Instruct"
device = "cuda" #Metal Performance Shaders
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="right") #padding_side show where padding tokens should be added to sequences
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name,
                                            torch_dtype=torch.bfloat16,
                                            device_map=device)

In [ ]:
generation_pipeline = pipeline(task="text-generation", #pipeline เชื่อมต่อโมเดลกับขั้นตอนอื่น ๆ
                               model=model, tokenizer=tokenizer)
generation_pipeline("Hello", max_new_tokens=25)

In [ ]:
prompts = [
    "hello how are you?",
    "The nigger"
]

tokenized = tokenizer(prompts,padding=True, return_tensors="pt").to(device)

print(tokenized)
print(tokenized["input_ids"].shape)

In [ ]:
tokenizer.batch_decode(tokenized["input_ids"])

In [ ]:
tokenized["attention_mask"] #tell which have value

In [ ]:
prompt_template = [
    {
      "role" : "system",
      "content" : "You are a smart AI assistant"
    },
    {
        "role" : "user",
        "content" : "Where does the sun rises?"
    },
    {
        "role" : "assistant",
        "content" : "Aye aye"
    }
]

tokenizer.pad_token = tokenizer.eos_token

tokenized = tokenizer.apply_chat_template( #converts prompt from chat message format to a single string sequence
    prompt_template,
    add_generation_prompt=True,
    tokenize=True, #if set True, will get number that easily insert to model
    padding=True,
    return_tensors="pt"
)

print(tokenized)

In [ ]:
out = model.generate(tokenized.to(device), max_new_tokens=30)

In [ ]:
decoded = tokenizer.batch_decode(out)
print(decoded[0])

In [ ]:
import torch.nn as nn

predict next word

In [ ]:
 sentence = "you are game"
 input_ids = tokenizer([sentence], return_tensors="pt")["input_ids"].to(device) # return_tensors use to change type to tensor from list

 out = model(input_ids = input_ids) # model(input) does one step of generation by outputting the probabilities of next word
 out

In [ ]:
# predict what is the next word
probability_dist = nn.Softmax(dim=-1)(out.logits[0, -1]) # softmax change to probability
print(out.logits)
print("_____________________________")
print(out.logits.shape) # models store millions of subwords, vocab has 128256 tokens
print("_____________________________")
print(input_ids)
print("_____________________________")
print(probability_dist)
print("_____________________________")
print(out.logits.argmax(), out.logits.argmax(axis=-1)) #argmax find maximum value in the list
print(tokenizer.decode(out.logits.argmax(axis=-1)[0, -1]))

loss function (training on sequence)

In [ ]:
sentence = ["use teng service instead."]
tokenized = tokenizer(sentence, return_tensors="pt")
print(tokenized)
tokenized2 = tokenizer(sentence, return_tensors="pt")["input_ids"] #print only input_ids
print(tokenized2)
print(tokenizer.batch_decode(tokenized2))

In [ ]:
input_ids = tokenized["input_ids"][:, :-1]
target_ids = tokenized["input_ids"][:, 1:]

print("Input Seq: ", input_ids)
print("Target Seq: ", tokenizer.batch_decode(input_ids))
print(target_ids)
print(tokenizer.batch_decode(target_ids))

In [ ]:
ques = "what service should I use?"
answer = "Teng Service"

prompt = [
    {"role": "user", "content": "what service should I use?"},
    {"role": "assistant", "content": "Service:"}
]
answer = "Teng Service"

chat_template = tokenizer.apply_chat_template(
    prompt,
    continue_final_message=True,
    tokenize=False
)
print(chat_template)

In [ ]:
full_response = chat_template + " " + answer + tokenizer.eos_token
print(full_response)

In [ ]:
tokenized = tokenizer(full_response, return_tensors="pt", add_special_tokens=False)["input_ids"]
print(tokenized)
print(tokenizer.batch_decode(tokenized))

In [ ]:
input_ids = tokenized[:, :-1]
target_ids = tokenized[:, 1:]

print("Input Seq: ", input_ids)
print("Target Seq: ", tokenizer.batch_decode(input_ids))
print(target_ids)
print(tokenizer.batch_decode(target_ids))

In [ ]:
# demonstrate of using tokenizer
print(tokenizer.convert_ids_to_tokens(32346))

In [ ]:
labels_tokenized = tokenizer([" " + answer], add_special_tokens=False, return_tensors="pt")["input_ids"] # answer in prompt
print(labels_tokenized)
print(tokenizer.batch_decode(labels_tokenized))

In [ ]:
input_ids.shape

In [ ]:
labels_tokenized2 = tokenizer([" " + answer + tokenizer.eos_token], add_special_tokens=False, return_tensors="pt", padding="max_length", max_length=target_ids.shape[1])["input_ids"]
print(labels_tokenized2)

In [ ]:
labels_tokenized_fixed = torch.where(labels_tokenized2 != tokenizer.pad_token_id, labels_tokenized2, -100)
labels_tokenized_fixed

In [ ]:
import torch # Assuming torch is used for tensor operations

def generate_input_output_pair(prompt, target_responses):
    chat_templates = tokenizer.apply_chat_template(prompt, continue_final_message=True, tokenize=False)
    full_response_text = [
        chat_template + " " + target_response + tokenizer.eos_token
        for chat_template, target_response in zip(chat_templates, target_responses)
    ]

    input_ids_tokenized = tokenizer(full_response_text, return_tensors="pt", add_special_tokens=False)["input_ids"]

    labels_tokenized = tokenizer([
        "" + response + tokenizer.eos_token for response in target_responses
    ], add_special_tokens=False, return_tensors="pt", padding="max_length", max_length=input_ids_tokenized.shape[1])["input_ids"]

    labels_tokenized_fixed = torch.where(labels_tokenized == tokenizer.pad_token_id, labels_tokenized, -100)
    labels_tokenized_fixed[:, -1] = tokenizer.pad_token_id

    input_ids_tokenized_left_shifted = input_ids_tokenized[:, :-1]
    labels_tokenized_right_shifted = labels_tokenized_fixed[:, 1:]

    attention_mask = input_ids_tokenized_left_shifted != tokenizer.pad_token_id

    return input_ids_tokenized_left_shifted, labels_tokenized_right_shifted, attention_mask

In [ ]:
import torch # Assuming torch is used for tensor operations

def generate_input_output_pair_test(prompt, target_responses):
    chat_templates = tokenizer.apply_chat_template(prompt, continue_final_message=True, tokenize=False)
    full_response_text = [
        chat_template + " " + target_response + tokenizer.eos_token
        for chat_template, target_response in zip(chat_templates, target_responses)
    ]

    input_ids_tokenized = tokenizer(full_response_text, return_tensors="pt", add_special_tokens=False)["input_ids"]

    labels_tokenized = tokenizer([
        "" + response + tokenizer.eos_token for response in target_responses
    ], add_special_tokens=False, return_tensors="pt", padding="max_length", max_length=input_ids_tokenized.shape[1])["input_ids"]

    # The original code was marking pad tokens in labels as -100, but also the last token
    # Let's revisit the logic for labels to ensure correct loss calculation.
    # For causal language modeling, the labels are typically the input IDs shifted by one position.
    # We only want to calculate the loss for the tokens corresponding to the target response.
    # Tokens before the target response and padding tokens should be masked with -100.

    # Create labels by shifting input_ids_tokenized
    labels = input_ids_tokenized.clone()
    # Shift labels to the left (the target for index i is the token at index i+1)
    labels[:, :-1] = input_ids_tokenized[:, 1:]
    # The last label should be -100 or the pad token id depending on desired behavior
    # For now, let's set it to -100 to match the original masking intent for non-response tokens
    labels[:, -1] = -100 # Assuming the last token should not contribute to the loss calculation for the label sequence

    # Create a mask to identify the tokens corresponding to the target response in the labels
    # This requires knowing where the target response starts in the full response text.
    # A more robust approach might be to tokenized the target response separately
    # and use its tokenized length to mask the labels tensor.

    # Let's try masking based on the tokenized target response length
    target_response_tokenized = tokenizer([response + tokenizer.eos_token for response in target_responses], add_special_tokens=False, return_tensors="pt")["input_ids"]
    target_response_length = target_response_tokenized.shape[1]

    # Mask the labels tensor. We want to calculate loss only for the target response tokens.
    # The tokens corresponding to the prompt and the continuation in chat_template should be masked.
    # The number of prompt tokens + continuation tokens is input_ids_tokenized.shape[1] - target_response_length
    prompt_and_continuation_length = input_ids_tokenized.shape[1] - target_response_length
    labels[:, :prompt_and_continuation_length] = -100 # Mask the prompt and continuation

    # Mask padding tokens in labels as -100
    labels[labels == tokenizer.pad_token_id] = -100


    # Ensure attention mask is correctly shaped for the shifted input
    attention_mask = input_ids_tokenized[:, :-1] != tokenizer.pad_token_id

    # Return a dictionary
    return {
        "input_ids": input_ids_tokenized[:, :-1], # Input is the sequence up to the second to last token
        "labels": labels[:, 1:], # Labels are the sequence from the second token onwards (shifted)
        "attention_mask": attention_mask
    }

# The training loop code remains the same after this change
# data = generate_input_output_pair(prompt=[training_prompt], target_responses=[target_response])
# data["input_ids"] = data["input_ids"].to(device)
# data["labels"] = data["labels"].to(device)

# optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)

# for i in range(10):
#   out = model(input_ids=data["input_ids"].to(device))
#   loss = calculate_loss(out.logits, data["labels"]).mean()

#   loss.backward()
#   optimizer.step()
#   optimizer.zero_grad()

In [ ]:
data = generate_input_output_pair(
    prompt = [
        [
          {"role": "user", "content": "what service should I use?"},
          {"role": "assistant", "content": "Service:"}
        ]
    ],
    target_responses = ["teng service"]
)

out = model(input_ids=data[0].to(device))
out

In [ ]:
print(data[0])
print(data[0].shape)
print(data[1])
print(data[1].shape)
print(data[2])
print(data[2].shape)

In [ ]:
def calculate_loss(logits, labels):
  loss_fn = nn.CrossEntropyLoss(reduction='none')
  cross_entropy_loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))
  return cross_entropy_loss

In [ ]:
out.logits.shape

In [ ]:
calculate_loss(out.logits, data[1].to(device))

In [ ]:
training_prompt = [
    {
        "role": "user", "content": "Who to subscribe to on YT for ML?"
    },
    {
        "role": "assistant", "content": "Subscribe to"
    }
]
target_response = "Neural Breaddown with AVB"

In [ ]:
test_tokenized = tokenizer.apply_chat_template(training_prompt, continue_final_message=True, return_tensors="pt").to(device)
test_out = model.generate(test_tokenized, max_new_tokens=10)
print(tokenizer.batch_decode(test_out, skip_special_tokens=True)[0])

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
import torch.nn as nn
from torch.optim import AdamW

In [ ]:
data = generate_input_output_pair(prompt=[training_prompt], target_responses=[target_response])
print(data)

In [ ]:
data = generate_input_output_pair_test(prompt=[training_prompt], target_responses=[target_response])
data["input_ids"] = data["input_ids"].to(device)
data["labels"] = data["labels"].to(device)

# optimizer update the weight of the network after the drop loss.
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)

for i in range(10):
  out = model(input_ids=data["input_ids"].to(device))
  loss = calculate_loss(out.logits, data["labels"]).mean()

  loss.backward()
  optimizer.step()
  optimizer.zero_grad()


In [ ]:
print(data)
print("-------------------------")
print(data["input_ids"])
print(data["input_ids"].shape)
print("-------------------------")
print(data["labels"])
print(data["labels"].shape)
print("-------------------------")
print(data["attention_mask"])
print(data["attention_mask"].shape)
print("-------------------------")

LORA


In [ ]:
if hasattr(model, 'peft_config'):
    model.unload()

In [ ]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()